Data 불러와서 나누기

In [148]:
import pickle
with open('./data/train_token.pickle', 'rb') as fs:
    train_token = pickle.load(fs)

with open('./data/dev_token.pickle', 'rb') as fs:
    dev_token = pickle.load(fs)

Data 탐색

데이터 복원 & 형태소 분석과 pos tagging

In [181]:
# from eunjeon import Mecab
# mecab = Mecab()
from konlpy.tag import Okt
okt = Okt()

doPos = lambda x: okt.pos(x)

train_sentence = list(map(' '.join, train_token))
dev_sentence = list(map(' '.join, dev_token))

pos_train = list(map(doPos, train_sentence))
pos_dev = list(map(doPos, dev_sentence))

In [150]:
pos_train[0]
pos_dev[0]

[('6일', 'Number'),
 ('유통업', 'Noun'),
 ('계', 'Suffix'),
 ('와', 'Josa'),
 ('정유', 'Noun'),
 ('업계', 'Noun'),
 ('에', 'Josa'),
 ('따르면', 'Verb'),
 ('‘', 'Foreign'),
 ('이마트', 'Noun'),
 ('-', 'Punctuation'),
 ('SK', 'Alpha'),
 ('’', 'Punctuation'),
 ('간판', 'Noun'),
 ('을', 'Josa'),
 ('내', 'Determiner'),
 ('걸', 'Noun'),
 ('ㄴ', 'KoreanParticle'),
 ('주유소', 'Noun'),
 ('가', 'Josa'),
 ('올해', 'Noun'),
 ('안', 'Noun'),
 ('에', 'Josa'),
 ('등장', 'Noun'),
 ('하', 'Suffix'),
 ('ㄹ', 'KoreanParticle'),
 ('것', 'Noun'),
 ('이', 'Josa'),
 ('확', 'Noun'),
 ('실시', 'Noun'),
 ('되자', 'Verb'),
 ('이마트', 'Noun'),
 ('와', 'Josa'),
 ('경쟁', 'Noun'),
 ('관계', 'Noun'),
 ('에', 'Josa'),
 ('있는', 'Adjective'),
 ('롯데', 'Noun'),
 ('마트', 'Noun'),
 ('홈플러스', 'Noun'),
 ('등', 'Noun'),
 ('다른', 'Noun'),
 ('대형', 'Noun'),
 ('마트', 'Noun'),
 ('도', 'Josa'),
 ('매장', 'Noun'),
 ('내', 'Noun'),
 ('주유소', 'Noun'),
 ('설립', 'Noun'),
 ('을', 'Josa'),
 ('위', 'Noun'),
 ('하', 'Suffix'),
 ('아', 'Josa'),
 ('정유', 'Noun'),
 ('사와', 'Verb'),
 ('물밑', 'Noun'),
 ('에서', 'J

B / I tagging 하기

In [153]:
def biPosTagging(lst):
    output = []
    for morph, pos in lst:
        B = "B_" + pos
        output.append(B)
        for i in range(1,len(morph)):
            I = "I_" + pos
            output.append(I)
    return output

bi_pos_train = []
bi_pos_dev = []

for lst in pos_train:
    bi_pos_train.append(biPosTagging(lst))

for lst in pos_dev:
    bi_pos_dev.append(biPosTagging(lst))

In [154]:
import pickle 
with open('./data/train_tag.pickle', 'rb') as fs:
    train_tag = pickle.load(fs)
with open('./data/dev_tag.pickle', 'rb') as fs:
    dev_tag = pickle.load(fs)

In [155]:
print(bi_pos_train[0])
print(train_tag[0])

print(bi_pos_dev[0])
print(dev_tag[0]

SyntaxError: unexpected EOF while parsing (<ipython-input-155-eec8141e2986>, line 5)

pos_vocab 만들기

In [ ]:
# pos_vocab for mecab
# pos_vocab = ['NNG','NNP','NNB','NNBC','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ','IC','JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EP','EF','EC','ETN','ETM','XPN','XSN','XSV','XSA','XR','SF','SE','SSO','SSC','SC','SY','SL','SH','SN']

In [ ]:
# pos_vocab for Okt
pos_vocab = ['Noun', 'Verb', 'Adjective', 'Determiner', 'Adverb', 'Conjunction', 'Exclamation', 'Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation', 'Foreign', 'Alpha', 'Number', 'Unknown', 'KoreanParticle', 'Hashtag', 'ScreenName', 'Email', 'URL']
# B-I tagging 하기
bi_pos_vocab = []
for pos in pos_vocab:
    B = 'B_' + pos 
    I = 'I_' + pos 
    bi_pos_vocab.append(B)
    bi_pos_vocab.append(I)

# train, dev 데이터에서 품사 집합 구해서 pos_vocab에 추가
pos_set_train = set([y for x in bi_pos_train for y in x])
pos_set_dev = set([y for x in bi_pos_dev for y in x])

pos_set = list(pos_set_train | pos_set_dev)

bi_pos_vocab = sorted(list(set(bi_pos_vocab) | set(pos_set)))
bi_pos_vocab.append('I_VerbPrefix')

In [ ]:
# pos_vocab 저장하기
with open('./data/pos_vocab.txt', 'w') as fs:
    fs.write('\n'.join((bi_pos_vocab)))

One_hot 벡터 만들기

In [ ]:
import numpy as np

def posOnehot(sentence):
    output = []

    for word in sentence:
        onehot = np.zeros(len(bi_pos_vocab))
        index = bi_pos_vocab.index(word)
        onehot[index] += 1 
        output.append(onehot)

    return output

train_onehot = list(map(posOnehot, bi_pos_train))
dev_onehot = list(map(posOnehot, bi_pos_dev))

In [183]:
with open('./data/train_onehot.pickle', 'wb') as fs:
    pickle.dump(train_onehot, fs)

with open('./data/dev_onehot.pickle', 'wb') as fs:
    pickle.dump(dev_onehot, fs)